In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import ee
import geemap
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

# 加载必要的数据

In [3]:
## 加载研究区域，以及设置缓冲区
# 加载研究区
EVSpencert = ee.Feature(ee.FeatureCollection('users/311605001111/region/EVSpencert').first())
Map.addLayer(EVSpencert, {}, "EVSpencert")
Map.centerObject(EVSpencert,12)

# 为研究区设定500m的缓冲区
EVSpencert_buffer = EVSpencert.buffer(500)
Map.addLayer(EVSpencert_buffer, {}, "EVSpencert_buffer")

In [4]:
landsat_images = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filterBounds(EVSpencert_buffer.geometry()) \
    .filterDate('2003-06-01', '2003-06-30') 
#     .map(cloudMaskL457)
visParams5 = {
  'bands': ['B3', 'B2', 'B1'],
  'min': 0,
  'max': 3000,
  'gamma': 1.4,
}
EVSpencert_landsat = landsat_images.mosaic().clip(EVSpencert_buffer.geometry())
Map.addLayer(EVSpencert_landsat,visParams5, 'EVSpencert_landsat')

In [5]:
## 加载月尺度产品和水体频率产品
# 加载月尺度水体制图产品
EVSpencert_monthly_water = ee.Image('JRC/GSW1_2/MonthlyHistory/2003_06').clip(EVSpencert_buffer)
visualization_monthly = {
  'bands': ['water'],
  'min': 0.0,
  'max': 2.0,
  'palette': ['green','fffcb8','0905ff']
}
Map.addLayer(EVSpencert_monthly_water,visualization_monthly, "EVSpencert_monthly_water")

# 加载water occurrence 制图产品 
EVSpencert_occurrence = ee.Image('JRC/GSW1_3/GlobalSurfaceWater').select('occurrence').clip(EVSpencert_buffer)
visualization_occurrence = {
  'bands': ['occurrence'],
  'min': 0.0,
  'max': 100.0,
  'palette': ['green', 'yellow', 'red']
}
Map.addLayer(EVSpencert_occurrence, visualization_occurrence, 'EVSpencert_occurrence')

In [6]:
roi = EVSpencert_buffer
# img指经过指数计算，但未做云掩膜的图像
def AutomaticCorrection_threshold(image):
    class_image = image.clip(roi)
    water = class_image.eq(2).selfMask().rename('waterclass')
    no_data = class_image.eq(0).selfMask()
    occurrence = ee.Image('JRC/GSW1_3/GlobalSurfaceWater').select('occurrence')
    occurrence_water = occurrence.updateMask(water)
    occurrence_no_data = occurrence.updateMask(no_data)
    occurrence_HistogramCount = occurrence_water.reduceRegion(**{
        'reducer': ee.Reducer.histogram(100,1),
        'geometry': roi.geometry(),
        'scale': 30,
        'bestEffort': True,
        'tileScale': 2,
    })
    histogram = ee.List(ee.Dictionary(occurrence_HistogramCount.get('occurrence')).get('histogram'))
    bucketMeans = ee.List(ee.Dictionary(occurrence_HistogramCount.get('occurrence')).get('bucketMeans'))    
    count_threshold = ee.Number(histogram.reduce(ee.Reducer.sum())).multiply(0.0017)
    index = histogram.map(lambda i : ee.Algorithms.If(ee.Number(i).gte(ee.Number(count_threshold)),ee.Number(i))).removeAll([None]).get(0)
    occurrence_threshold = bucketMeans.get(histogram.indexOf(index))
    print(occurrence_threshold.getInfo())
    return image.set({'occurrence_threshold':occurrence_threshold})
a = AutomaticCorrection_threshold(EVSpencert_monthly_water)
# Map.addLayer(a,visParams5, 'a')

61


In [ ]:
roi = EVSpencert_buffer
def AutomaticCorrection(image):
    basemap = ee.Image.constant(0).toFloat().clip(image.geometry()).clip(roi).rename('waterclass')
    class_image = image.clip(roi)
    water = class_image.eq(2).selfMask().rename('waterclass')
    no_data = class_image.eq(0).selfMask()
    occurrence = ee.Image('JRC/GSW1_3/GlobalSurfaceWater').select('occurrence').clip(image.geometry()).clip(roi)
    occurrence_water = occurrence.updateMask(water)
    occurrence_no_data = occurrence.updateMask(no_data)
    # Map.addLayer(occurrence_water, visualization_occurrence, 'water in occurrence')
    # Map.addLayer(occurrence_no_data, visualization_occurrence, 'no data in occurrence')
    occurrence_HistogramCount = occurrence_water.reduceRegion(**{
        'reducer': ee.Reducer.histogram(100,1),
        'geometry': image.clip(roi).geometry(),
        'scale': 30,
    })
    histogram = ee.List(ee.Dictionary(occurrence_HistogramCount.get('occurrence')).get('histogram'))
    bucketMeans = ee.List(ee.Dictionary(occurrence_HistogramCount.get('occurrence')).get('bucketMeans'))    
    count_threshold = ee.Number(histogram.reduce(ee.Reducer.sum())).multiply(0.0017)
    index = histogram.map(lambda i : ee.Algorithms.If(ee.Number(i).gte(ee.Number(count_threshold)),ee.Number(i))).removeAll([None]).get(0)
    occurrence_threshold = bucketMeans.get(histogram.indexOf(index))
    print(occurrence_threshold.getInfo())
    occurrence_corrected_water = occurrence_no_data.gte(ee.Number(occurrence_threshold)).selfMask().select('occurrence').rename('waterclass')
    enhanced_water = ee.ImageCollection([basemap,water,occurrence_corrected_water]).sum()    
    return enhanced_water

a = AutomaticCorrection(EVSpencert_monthly_water)
Map.addLayer(a.selfMask(),{'palette': ['blue']}, 'EVSpencert_enhanced_water')

In [ ]:
# 确定无效像元和水体的范围
EVSpencert_water = EVSpencert_monthly_water.eq(2).selfMask()
EVSpencert_no_data = EVSpencert_monthly_water.eq(0).selfMask()
# Map.addLayer(EVSpencert_water,{'palette': ['cyan']}, 'EVSpencert_water')

# 在水体频率上，确定无效像元和水体的范围
occurrence_water = EVSpencert_occurrence.updateMask(EVSpencert_water)
Map.addLayer(occurrence_water, visualization_occurrence, 'water in occurrence')
occurrence_no_data = EVSpencert_occurrence.updateMask(EVSpencert_no_data)
Map.addLayer(occurrence_no_data, visualization_occurrence, 'no data in occurrence')

# 确定 count threshold 阈值

In [ ]:
# 计算 count threshold ；count_threshold = "The average value of pixel counts" *0.17
occurrence_HistogramCount = occurrence_water.reduceRegion(**{
    'reducer': ee.Reducer.histogram(100,1),
    'geometry': EVSpencert_buffer.geometry(),
    'scale': 30,
})
histogram = ee.List(ee.Dictionary(occurrence_HistogramCount.get('occurrence')).get('histogram'))
bucketMeans = ee.List(ee.Dictionary(occurrence_HistogramCount.get('occurrence')).get('bucketMeans'))

In [ ]:
HistogramCount_sum = histogram.reduce(ee.Reducer.sum())
print(histogram.getInfo())
count_threshold = ee.Number(HistogramCount_sum).multiply(0.0017)
print("The sum of pixel counts : {}".format(HistogramCount_sum.getInfo()))
print("count threshold : {}".format(count_threshold.getInfo()))

# 确定 occurrence threshold 阈值

方法： 执行下面方框中的代码，将出现一个2维数组，其含义为：第一列为 water frequency, 第二列为 the pixel counts in this frequency

例如：[5,2]表示为：水体频率为5的像元个数是2； [99，41]表示为：水体频率为99的像元个数是41。

查找到第一个 pixel count 大于或等于 count threshold 的情况， 该 pixel count 所对应的 water frequency 即为 occurrence threshold。

In [ ]:
index = histogram.map(lambda i : ee.Algorithms.If(ee.Number(i).gte(ee.Number(count_threshold)),ee.Number(i))).removeAll([None]).get(0)
occurrence_threshold = bucketMeans.get(histogram.indexOf(index))
print(occurrence_threshold.getInfo())

In [ ]:
def AutomaticCorrection(image):
    basemap = ee.Image.constant(0).toFloat().clip(image.geometry()).clip(roi).rename('waterclass')
    class_image = ee.ImageCollection([basemap,image.remap([0,1],[1,2]).rename('waterclass')]).sum()
    water = class_image.eq(2).selfMask()
    no_data = class_image.eq(0).selfMask()
    occurrence = ee.Image('JRC/GSW1_3/GlobalSurfaceWater').select('occurrence').clip(image.geometry()).clip(roi)
    occurrence_water = occurrence.updateMask(water)
    occurrence_no_data = occurrence.updateMask(no_data)
    occurrence_HistogramCount = occurrence_water.reduceRegion(**{
        'reducer': ee.Reducer.histogram(100,1),
        'geometry': image.clip(roi).geometry(),
        'scale': 30,
    })
    histogram = ee.List(ee.Dictionary(occurrence_HistogramCount.get('occurrence')).get('histogram'))
    bucketMeans = ee.List(ee.Dictionary(occurrence_HistogramCount.get('occurrence')).get('bucketMeans'))    
    count_threshold = ee.Number(histogram.reduce(ee.Reducer.sum())).multiply(0.0017)
    index = histogram.map(lambda i : ee.Algorithms.If(ee.Number(i).gte(ee.Number(count_threshold)),ee.Number(i))).removeAll([None]).get(0)
    occurrence_threshold = bucketMeans.get(histogram.indexOf(index))
    print(occurrence_threshold.getInfo())
    occurrence_corrected_water = occurrence_no_data.gte(ee.Number(occurrence_threshold)).selfMask().select('occurrence').rename('waterclass')
    enhanced_water = ee.ImageCollection([basemap,water,occurrence_corrected_water]).sum()
    return enhanced_water

In [ ]:
# occurrence_histogram = occurrence_water.reduceRegion(**{
#     'reducer': ee.Reducer.autoHistogram(100,1),
#     'geometry': EVSpencert_buffer.geometry(),
#     'scale': 30,
# })

# def lookFrequency(array):
#     counts = ee.Number(ee.List(array).get(1))
#     a = ee.Algorithms.If(counts.gte(ee.Number(count_threshold)),ee.Number(ee.List(array).get(0)))
#     return a
# occurrence_list = ee.List(occurrence_histogram.get('occurrence').getInfo())
# occurrence_threshold = occurrence_list.map(lookFrequency).removeAll([None]).get(0)
# print(occurrence_threshold.getInfo())

# 根据occurrence threshold , 求得增强后的湖泊范围

In [ ]:
# 第一行代码中的“58”为 occcurrence threshold

occurrence_corrected_water = occurrence_no_data.gte(ee.Number(occurrence_threshold)).selfMask().select('occurrence').rename('water')
Map.addLayer(occurrence_corrected_water,{'palette': ['cyan']}, 'corrected waterbody')

EVSpencert_enhanced_water = ee.ImageCollection([EVSpencert_water,occurrence_corrected_water]).sum()
Map.addLayer(EVSpencert_enhanced_water,{'palette': ['blue']}, 'EVSpencert_enhanced_water')

# 制图

In [ ]:
def cloudMaskL457(image):
    qa = image.select('pixel_qa')
    cloud = qa.bitwiseAnd(1 << 5) \
                   .And(qa.bitwiseAnd(1 << 7)) \
                   .Or(qa.bitwiseAnd(1 << 3))
    mask2 = image.mask().reduce(ee.Reducer.min())
    return image.updateMask(cloud.Not()).updateMask(mask2)

visParams5 = {
  'bands': ['B3', 'B2', 'B1'],
  'min': 0,
  'max': 3000,
  'gamma': 1.4,
}

In [ ]:
landsat_images = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filterBounds(EVSpencert_buffer.geometry()) \
    .filterDate('2003-06-01', '2003-06-30') 
#     .map(cloudMaskL457)
EVSpencert_landsat = landsat_images.mosaic().clip(EVSpencert_buffer.geometry())
Map.addLayer(EVSpencert_landsat,visParams5, 'EVSpencert_landsat')

In [ ]:
# 制图
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [-100.595, 31.87, -100.51, 31.951] 
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(EVSpencert_landsat, region=region, vis_params=visParams5)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.02,0.02], linestyle=":")
ax.set_title(label = 'the landsat5(EVSpencert) in 2003/06', fontsize=20)

In [ ]:
# 制图
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [-100.595, 31.87, -100.51, 31.951] 
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(EVSpencert_monthly_water, region=region, vis_params=visualization_monthly)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.02,0.02], linestyle=":")
ax.set_title(label = 'EVSPencert waterbody in 2003/06', fontsize=20)

In [ ]:
# 制图
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [-100.595, 31.87, -100.51, 31.951] 
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(EVSpencert_occurrence, region=region, vis_params=visualization_occurrence)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.02,0.02], linestyle=":")
ax.set_title(label = 'water occurrence in EVSpencert', fontsize=20)

In [ ]:
# 制图
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [-100.595, 31.87, -100.51, 31.951] 
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(occurrence_water, region=region, vis_params=visualization_occurrence)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.03,0.03], linestyle=":")
ax.set_title(label = 'water occurrence in waterbody region', fontsize=20)

In [ ]:
# 制图
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [-100.595, 31.87, -100.51, 31.951] 
vis = {'bands': ['water'], 'palette': ['blue'], 'min': 0.0, 'max': 1.0, 'opacity': 1.0}
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(EVSpencert_enhanced_water, region=region, vis_params=vis)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.02,0.02], linestyle=":")
ax.set_title(label = 'enhanced water in EVSPencert', fontsize=20)

In [ ]:
# 制图
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [-100.595, 31.87, -100.51, 31.951] 
vis = {'bands': ['water'], 'palette': ['blue'], 'min': 0.0, 'max': 1.0, 'opacity': 1.0}
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(occurrence_corrected_water, region=region, vis_params=vis)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.02,0.02], linestyle=":")
ax.set_title(label = 'occurrence_corrected_water', fontsize=20)